In [24]:
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract
import webbrowser
import pyscreenshot as ImageGrab
from googleapiclient.discovery import build
import pprint
import cPickle as pickle

In [27]:
my_api_key, my_cse_id = pickle.load(open('api_key_cse_id.p', 'rb'))

def google_search(search_term, **kwargs):
    service = build("customsearch", "v1", developerKey=my_api_key)
    res = service.cse().list(q=search_term, cx=my_cse_id, **kwargs).execute()
    return res['items']

def print_results(results):
    for result in results:
        for key in result:
            print '%s: %s' % (key, result[key])
        print '\n\n'

In [14]:
''' coordinates for Reflector 3 window mirroring iPhone at "Fill Screen" scale,
positioned at (0, 0) on 13 inch MBA '''

question_coords = (55, 253, 376, 357)

answerX1, answerX2 = 75, 355
answerY1, answerY2 = 390, 415
offset = 63
answers_coords = [(answerX1, answerY1, answerX2, answerY2), 
                  (answerX1, answerY1 + offset, answerX2, answerY2 + offset),
                  (answerX1, answerY1 + 2*offset, answerX2, answerY2 + 2*offset)]

def parseQuestionAnswer():
    question_img=ImageGrab.grab(bbox=question_coords) # X1,Y1,X2,Y2
    answer_imgs = [ImageGrab.grab(bbox=coords) for coords in answers_coords]
    question = pytesseract.image_to_string(question_img).replace('\n', ' ')
    answers = [pytesseract.image_to_string(img) for img in answer_imgs]
    return question, answers

(u'Which of these is a species of snake?', [u'Ball python', u'Marbled salamander', u'Axolotl'])


In [28]:
question, answers = parseQuestionAnswer()
results = google_search(question, num=10)
print_results(results)

kind: customsearch#result
title: These 5 New Snake Species Suck Snails Out of Their Shells
displayLink: news.nationalgeographic.com
htmlTitle: <b>These</b> 5 New <b>Snake Species</b> Suck Snails Out of Their Shells
formattedUrl: https://news.nationalgeographic.com/.../snakes-new-species-ecuador-animals/
htmlFormattedUrl: https://news.nationalgeographic.com/.../<b>snakes</b>-new-<b>species</b>-ecuador-animals/
pagemap: {u'imagegallery': [{u'caption': u'A red bamboo snake, Oreocryptophis porphyraceus vaillantii, from a private collection.\u2026 Read More', u'author': u'Photograph by Joel Sartore, National Geographic Photo Ark'}], u'webpage': [{u'maincontentofpage': u'Photograph by Alejandro ArteagaRead CaptionWith slate-blue eyes, the research team says Sibon bevridgelyi is the prettiest snail-sucker of the lot. Photograph by Alejandro ArteagaWeird & WildThese...'}], u'metatags': [{u'twitter:creator': u'@NatGeo', u'og:image:height': u'1266', u'tax:locations': u'ecuador south_america', u'